# Supervised Fine-Tuning with SFTTrainer

This notebook demonstrates how to fine-tune the `HuggingFaceTB/SmolLM2-135M` model using the `SFTTrainer` from the `trl` library. The notebook cells run and will finetune the model. You can select your difficulty by trying out different datasets.

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Exercise: Fine-Tuning SmolLM2 with SFTTrainer</h2>
    <p>Take a dataset from the Hugging Face hub and finetune a model on it. </p> 
    <p><b>Difficulty Levels</b></p>
    <p>🐢 Use the `HuggingFaceTB/smoltalk` dataset</p>
    <p>🐕 Try out the `bigcode/the-stack-smol` dataset and finetune a code generation model on a specific subset `data/python`.</p>
    <p>🦁 Select a dataset that relates to a real world use case your interested in</p>
</div>

In [2]:
# # Install the requirements in Google Colab
# # !pip install transformers datasets trl huggingface_hub

# # Authenticate to Hugging Face

# from huggingface_hub import login
# login()

# # for convenience you can create an environment variable containing your hub token as HF_TOKEN

- `Base-Model` -> `SFT-Model`
- `Instructed-Model` -> `Reasoning-Model`

In [1]:
# Import necessary libraries
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, setup_chat_format
import torch

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

# Load the model and tokenizer
model_name = "HuggingFaceTB/SmolLM2-135M"
# model_name = "HuggingFaceTB/SmolLM2-135M-Instruct"
# model_name = "Qwen/Qwen2.5-0.5B"
# model_name = "Qwen/Qwen2.5-0.5B-Instruct"
# r1_model_name = "open-r1/OpenR1-Qwen-7B"
# r1_model_name = "HuggingFaceTB/SmolLM2-135M-Instruct"
# r1_model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"

model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name
).to(device)

tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)
r1_tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

# Set up the chat format
# model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)

# Set our name for the finetune to be saved &/ uploaded to
# finetune_name = "SmolLM2-FT-MyDataset"
# finetune_tags = ["smol-course", "module_1"]

In [4]:
tokenizer.special_tokens_map

{'bos_token': '<|endoftext|>',
 'eos_token': '<|endoftext|>',
 'unk_token': '<|endoftext|>',
 'additional_special_tokens': ['<|endoftext|>',
  '<|im_start|>',
  '<|im_end|>',
  '<repo_name>',
  '<reponame>',
  '<file_sep>',
  '<filename>',
  '<gh_stars>',
  '<issue_start>',
  '<issue_comment>',
  '<issue_closed>',
  '<jupyter_start>',
  '<jupyter_text>',
  '<jupyter_code>',
  '<jupyter_output>',
  '<jupyter_script>',
  '<empty_output>']}

In [ ]:
chat_template = """{% for message in messages %}{% if loop.first and messages[0]['role'] != 'system' %}{{ '<|im_start|>system
You are a helpful AI assistant named MKLLM, trained by mikeluck<|im_end|>
' }}{% endif %}{{'<|im_start|>' + message['role'] + '
' + message['content'] + '<|im_end|>' + '
'}}{% endfor %}{% if add_generation_prompt %}{{ '<|im_start|>assistant
' }}{% endif %}"""

r1_tokenizer.chat_template = chat_template
print(tokenizer.chat_template) #base model tokenizer is none
print("#"*200)
print(r1_tokenizer.chat_template) #replace instructed model chat template

In [2]:
# print(f"Before extended tokenizer size: {len(tokenizer)}")
# print(f"Before token embedding size: {model.config.vocab_size}")
# model
# n_params = 0
# for name, params in model.named_parameters():
#     n_params += params.numel()
#     print(f"{name}: {params.size()}, - {params.numel():,}")
# print(f"Total Parameters: {n_params:,}")
# tokenizer.all_special_tokens, tokenizer.special_tokens_map

In [13]:
# # extend special tokens
# special_tokens_dict = {"additional_special_tokens": tokenizer.special_tokens_map['additional_special_tokens'] + ["<|think|>", "<|think_end|>"]}
# tokenizer.add_special_tokens(special_tokens_dict)
# model.resize_token_embeddings(len(tokenizer)) # Caution!!!
# print(f"After extended tokenizer size: {len(tokenizer)}")
# print(f"After token embedding size: {model.vocab_size}")


# Generate with the base model

Here we will try out the base model which does not have a chat template. 

In [ ]:
# base model tokenizer
prompt = "你是谁？"
inputs = tokenizer.encode(prompt, return_tensors='pt').to(model.device)

outputs = model.generate(inputs, max_new_tokens=128)
print(tokenizer.decode(outputs[0]))

In [ ]:
# sft tokenizer
messages = [
    {"role": "user", "content": "你是谁？"}
]
formatted_prompt = r1_tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
print(formatted_prompt)

inputs = r1_tokenizer(formatted_prompt, return_tensors='pt').to(model.device)
outputs = model.generate(**inputs, max_new_tokens=512)

print(tokenizer.decode(outputs[0]))

## Dataset Preparation

We will load a sample dataset and format it for training. The dataset should be structured with input-output pairs, where each input is a prompt and the output is the expected response from the model.

**TRL will format input messages based on the model's chat templates.** They need to be represented as a list of dictionaries with the keys: `role` and `content`,.

In [ ]:
# Load a sample dataset
from datasets import load_dataset

datasets_name = "HuggingFaceTB/smoltalk"
# datasets_name = "O1-OPEN/OpenO1-SFT"
# datasets_name = "open-r1/OpenR1-Math-220k"

ds = load_dataset(path=datasets_name, name='everyday-conversations')
# ds = load_dataset(path=datasets_name, split='default')


def process_dataset(sample):
    content = r1_tokenizer.apply_chat_template(
        sample['messages'], tokenize=False
    )
    return {"content": content}

# def process_dataset(sample):
#     messages = [
#         {"role": "system", "content": "You are MKLLM, created by Mike_Luck. You are a helpful assistant."},
#         {"role": "user", "content": sample['instruction']},
#         {"role": "assistant", "content": sample['output'].replace('<Thought>', '<think>').replace('</Thought>', '</think>')},
#     ]
#     sample = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
#     return {"content": sample}

ds = ds.map(process_dataset)
# ds = ds['train'].train_test_split(test_size=0.1, shuffle=True) # 分割训练集为训练和测试
# ds = ds.train_test_split(test_size=0.1, shuffle=True)
ds

In [ ]:
# ds['train'][0]['content']

In [ ]:
# TODO: 🦁 If your dataset is not in a format that TRL can convert to the chat template, you will need to process it. Refer to the [module](../chat_templates.md)

## Configuring the SFTTrainer

The `SFTTrainer` is configured with various parameters that control the training process. These include the number of training steps, batch size, learning rate, and evaluation strategy. Adjust these parameters based on your specific requirements and computational resources.

In [ ]:
# Configure the SFTTrainer
sft_config = SFTConfig(
    dataset_text_field='content',
    output_dir="./sft_output",
    # max_steps=1000,  # Adjust based on dataset size and desired training duration
    num_train_epochs=3,
    per_device_train_batch_size=4,  # Set according to your GPU memory capacity
    per_device_eval_batch_size=4,
    learning_rate=5e-5,  # Common starting point for fine-tuning
    logging_steps=10,  # Frequency of logging training metrics
    save_steps=200,  # Frequency of saving model checkpoints
    eval_strategy="steps",  # Evaluate the model at regular intervals
    eval_steps=50,  # Frequency of evaluation
    use_mps_device=(
        True if device == "mps" else False
    ),  # Use MPS for mixed precision training
    # hub_model_id=finetune_name,  # Set a unique name for your model
    report_to='tensorboard'
)

# Initialize the SFTTrainer
trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=ds["train"],
    tokenizer=r1_tokenizer,
    eval_dataset=ds["test"],
)

# TODO: 🦁 🐕 align the SFTTrainer params with your chosen dataset. For example, if you are using the `bigcode/the-stack-smol` dataset, you will need to choose the `content` column`

## Training the Model

With the trainer configured, we can now proceed to train the model. The training process will involve iterating over the dataset, computing the loss, and updating the model's parameters to minimize this loss.

In [ ]:
# Train the model
trainer.train()

finetune_name = 'sft_demo'
# Save the model
trainer.save_model(f"./{finetune_name}")

In [ ]:
trainer.push_to_hub(tags=finetune_tags)

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Bonus Exercise: Generate with fine-tuned model</h2>
    <p>🐕 Use the fine-tuned to model generate a response, just like with the base example..</p>
</div>

In [ ]:
# Test the fine-tuned model on the same prompt

# Let's test the base model before training
prompt = "Write a haiku about programming"

# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Generate response
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)

# TODO: use the fine-tuned to model generate a response, just like with the base example.

## 💐 You're done!

This notebook provided a step-by-step guide to fine-tuning the `HuggingFaceTB/SmolLM2-135M` model using the `SFTTrainer`. By following these steps, you can adapt the model to perform specific tasks more effectively. If you want to carry on working on this course, here are steps you could try out:

- Try this notebook on a harder difficulty
- Review a colleagues PR
- Improve the course material via an Issue or PR.